In [1]:
pip install snake-egg

ERROR: Could not find a version that satisfies the requirement snake-egg (from versions: none)
ERROR: No matching distribution found for snake-egg
Note: you may need to restart the kernel to use updated packages.
